<a href="https://colab.research.google.com/github/johyunkang/python_stat/blob/main/chapter12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 12 회귀분석

p. 372